In [1]:
import torch
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# sys.argv: [脚本名, 设备, 精度]
sys.argv = ['notebook', 'cuda' if torch.cuda.is_available() else 'cpu', 'fp32'] #'fp16' #'fp32'# 'bf16' #
device = sys.argv[1]
sys_dtype = sys.argv[2] if len(sys.argv) > 2 else 'fp32'

print('Using device:', device)
print('Using dtype:', sys_dtype)

# dr settings
NUM_CONF=5
MAX_LENGTH=480
MAX_CAT_LENGTH=2400

CFG_DIR='cfg_97'
CFG_MERGE=False
DR_SCORE=False
OP_SCORE=True

NO_SORT=False
GET_CENTER=False

FULL_ENERGY=True

OPTIM_LENGTH=0

DEVICE=device #'cuda' #'cpu'#
PREC=sys_dtype

# ARENA='./Arena/Arena'
ARENA_DIR='/kaggle/input/drfold-model-bf16/Arena'

Using device: cuda
Using dtype: fp32


In [2]:
import shutil
dst = os.path.join(os.getcwd(), 'Arena')  # 拷贝到当前工作目录下的 ./Arena

# 如果目标已存在，就先删掉，保证每次都是全新的拷贝
if os.path.exists(dst):
    shutil.rmtree(dst)

shutil.copytree(ARENA_DIR, dst)
print(f"Arena 目录已拷贝到：{dst}")
arena_exec = os.path.join(dst, 'Arena')      # 二进制文件路径
os.chmod(arena_exec, 0o755)                 # 增加可执行权限

Arena 目录已拷贝到：/kaggle/working/Arena


In [3]:
if PREC=='fp16':
    torch.set_default_dtype(torch.float16)
if PREC=='bf16':
    torch.set_default_dtype(torch.bfloat16)

## Install requirements 

In [4]:
#rhofold+
#!pip install --no-deps '/kaggle/input/rhofold/wheels_py311/python_box-7.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
!pip install --no-deps '/kaggle/input/rhofold/wheels_py311/gprofiler_official-1.0.0-py3-none-any.whl'
!pip install --no-deps '/kaggle/input/rhofold/wheels_py311/mygene-3.2.2-py2.py3-none-any.whl'
!pip install --no-deps '/kaggle/input/rhofold/wheels_py311/biothings_client-0.4.1-py3-none-any.whl'
#!pip install --no-deps '/kaggle/input/rhofold/wheels_py311/dm_tree-0.1.9-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
!pip install --no-deps '/kaggle/input/rhofold/wheels_py311/einops-0.8.1-py3-none-any.whl'
#!pip install --no-deps '/kaggle/input/rhofold/wheels_py311/openmm-7.7.0-py3-none-any.whl'
!pip install --no-deps '/kaggle/input/rhofold/wheels_py311/bio-1.8.0-py3-none-any.whl'
!pip install --no-deps '/kaggle/input/rhofold/wheels_py311/absl_py-2.2.2-py3-none-any.whl'
!pip install --no-deps '/kaggle/input/rhofold/wheels_py311/OpenMM-8.2.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl'
print("-------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------")

!pip install --no-deps '/kaggle/input/dependencies-tr-pr/protenix-0.4.6-py3-none-any.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/rdkit-2024.9.6-cp310-cp310-manylinux_2_28_x86_64.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/ml_collections-1.1.0-py3-none-any.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/biopython-1.85-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/pyrosetta-2025.13-cp310-cp310-linux_x86_64.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/blosc-1.11.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/ml_collections-1.1.0-py3-none-any.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/biotraj-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/biotite-1.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/biopandas-0.5.1-py3-none-any.whl'
!pip install --no-deps '/kaggle/input/dependencies-tr-pr/looseversion-1.1.2-py3-none-any.whl'


Processing /kaggle/input/rhofold/wheels_py311/gprofiler_official-1.0.0-py3-none-any.whl
Processing /kaggle/input/rhofold/wheels_py311/mygene-3.2.2-py2.py3-none-any.whl
Processing /kaggle/input/rhofold/wheels_py311/biothings_client-0.4.1-py3-none-any.whl
Processing /kaggle/input/rhofold/wheels_py311/einops-0.8.1-py3-none-any.whl
  Attempting uninstall: einops
    Found existing installation: einops 0.8.0
    Uninstalling einops-0.8.0:
      Successfully uninstalled einops-0.8.0
Processing /kaggle/input/rhofold/wheels_py311/bio-1.8.0-py3-none-any.whl
Processing /kaggle/input/rhofold/wheels_py311/absl_py-2.2.2-py3-none-any.whl
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
Processing /kaggle/input/rhofold/wheels_py311/OpenMM-8.2.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl
-------------------------------------------------------------------------------
-------------

In [5]:
!python --version

Python 3.10.12


# Drfold2

In [6]:
from datetime import datetime
import pytz
print('LOGGING TIME OF START:',  datetime.strftime(datetime.now(pytz.timezone('Asia/Singapore')), "%Y-%m-%d %H:%M:%S"))


print('PIP INSTALL OK !!!!')
import os,sys

import pandas as pd
pd.set_option('display.max_columns', 20)
pd.set_option('display.expand_frame_repr', False)

import numpy as np
import torch
import torch.nn.functional as F
from timeit import default_timer as timer



# helper--
class dotdict(dict):
	__setattr__ = dict.__setitem__
	__delattr__ = dict.__delitem__

	def __getattr__(self, name):
		try:
			return self[name]
		except KeyError:
			raise AttributeError(name)

def time_to_str(t, mode='min'):
	if mode=='min':
		t  = int(t)/60
		hr = t//60
		min = t%60
		return '%2d hr %02d min'%(hr,min) 
	elif mode=='sec':
		t   = int(t)
		min = t//60
		sec = t%60
		return '%2d min %02d sec'%(min,sec)

	else:
		raise NotImplementedError

def gpu_memory_use():
    if torch.cuda.is_available():
        device = torch.device(0)
        free, total = torch.cuda.mem_get_info(device)
        used= (total - free) / 1024 ** 3
        return round(used,2)
    else:
        return 0

def set_aspect_equal(ax):
	x_limits = ax.get_xlim()
	y_limits = ax.get_ylim()
	z_limits = ax.get_zlim()

	# Compute the mean of each axis
	x_middle = np.mean(x_limits)
	y_middle = np.mean(y_limits)
	z_middle = np.mean(z_limits)

	# Compute the max range across all axes
	max_range = max(x_limits[1] - x_limits[0],
					y_limits[1] - y_limits[0],
					z_limits[1] - z_limits[0]) / 2.0

	# Set the new limits to ensure equal scaling
	ax.set_xlim(x_middle - max_range, x_middle + max_range)
	ax.set_ylim(y_middle - max_range, y_middle + max_range)
	ax.set_zlim(z_middle - max_range, z_middle + max_range)


print('torch',torch.__version__)
print('torch.cuda',torch.version.cuda)

print('IMPORT OK!!!')
MODE = 'submit' #'local' # submit

DATA_KAGGLE_DIR = '/kaggle/input/stanford-rna-3d-folding'

if MODE == 'local':
    valid_df = pd.read_csv(f'{DATA_KAGGLE_DIR}/validation_sequences.csv')
    label_df = pd.read_csv(f'{DATA_KAGGLE_DIR}/validation_labels.csv')
    label_df['target_id'] = label_df['ID'].apply(lambda x: '_'.join(x.split('_')[:-1]))

if MODE == 'submit':
	valid_df = pd.read_csv(f'{DATA_KAGGLE_DIR}/test_sequences.csv')

print('len(valid_df)',len(valid_df))
print(valid_df.iloc[0])
print('')


print('MODE:', MODE)
print('SETTING OK!!!')


LOGGING TIME OF START: 2025-05-24 21:20:34
PIP INSTALL OK !!!!
torch 2.5.1+cu121
torch.cuda 12.1
IMPORT OK!!!
len(valid_df) 12
target_id                                                      R1107
sequence           GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUU...
temporal_cutoff                                           2022-05-28
description        CPEB3 ribozyme\nHuman\nhuman CPEB3 HDV-like ri...
all_sequences      >7QR4_1|Chain A|U1 small nuclear ribonucleopro...
Name: 0, dtype: object

MODE: submit
SETTING OK!!!


In [7]:
sys.path.append('/kaggle/input/drfold-model-bf16/PotentialFold')
import a2b
def frame_coor_to_C1(coor, seq, BASE_COOR, OTHER_COOR):
    """
    使用四元数批量运算将模型输出的局部 frame 坐标直接转换为每个位点 C1' 原子全局坐标。

    参数
    ----
    coor : numpy.ndarray 或 torch.Tensor, shape=(L, 3, 3)
        模型预测的每个位点 P, C4', N 三点的局部坐标
    seq : str, length=L
        核苷酸序列
    BASE_COOR : dict
        Get_base 所需的三个骨架原子模板坐标常量
    OTHER_COOR : dict
        Get_base 所需的五个糖环原子模板坐标常量

    返回
    ----
    c1_xyz : numpy.ndarray, shape=(L, 3)
        每个位点 C1' 原子在全局坐标系下的 XYZ
    """
    # 1. 转为 torch.Tensor
    tx = torch.as_tensor(coor, dtype=torch.float32)
    basex = torch.from_numpy(Get_base(seq, BASE_COOR)).to(tx.dtype)
    otherx = torch.from_numpy(Get_base(seq, OTHER_COOR)).to(tx.dtype)
    L = len(seq)

    # 2. 初始化参数向量 x：21 维，其中前 18 维用于旋转投影，后 3 维用于平移
    x = torch.rand((L, 21), dtype=tx.dtype, device=tx.device)

    # 3. 中心化处理：计算质心 biasq，并得到零均值坐标 q
    biasq = tx.mean(dim=1, keepdim=True)            # (L, 1, 3)
    q = tx - biasq                                  # (L, 3, 3)

    # 4. 计算旋转投影 m，并填充到 x 的前两组 9 维
    m = torch.einsum('bnz,bny->bzy', basex, q).reshape(L, -1)
    x[:, :9] = m
    x[:, 9:18] = m

    # 5. 填入平移参数（将 biasq 作为平移量）
    x[:, 18:] = biasq.squeeze(1)
    rama = x.double()  # 四元数参数需要 double 精度

    # 6. 调用 quat2b：对模板中所有糖环原子做旋转+平移
    #    otherx: (L, 5, 3) -> other_xyz: (L, 5, 3)
    other_xyz = a2b.quat2b(otherx.double(), rama[:, 9:]).float().cpu().numpy()

    # 7. 提取第 5 个原子（C1'）并返回
    c1_xyz = other_xyz[:, 4, :]
    return c1_xyz


def concat_coor(out1: dict, out2: dict) -> np.ndarray:
    """
    将 out2 通过四元数（quat2b）做旋转 + 平移对齐到 out1 末尾，并拼接（去掉重复的第一个碱基）。
    out1/out2 都是 dict，包含
      'coor': np.ndarray, shape=(L, 3, 3)  —— 每个碱基的 P, C4, N 坐标
    返回拼接后的坐标数组，shape = (L1 + L2 - 1, 3, 3)
    """
    # 1. 转为 torch.DoubleTensor
    coor1 = torch.as_tensor(out1['coor'], dtype=torch.float64)   # (L1,3,3)
    coor2 = torch.as_tensor(out2['coor'], dtype=torch.float64)   # (L2,3,3)

    # 2. 取公共碱基的 frame：out1 最后一个、out2 第一个
    f1 = coor1[-1]   # (3,3)
    f2 = coor2[0]    # (3,3)

    # 3. 中心化：去除各自质心
    bias1 = f1.mean(dim=0)   # (3,)
    bias2 = f2.mean(dim=0)   # (3,)
    basex = f1 - bias1       # (3,3)
    q     = f2 - bias2       # (3,3)

    # 4. 用 einsum 计算旋转矩阵 R = basex · q^T
    #    R_{ij} = sum_z basex_{iz} * q_{jz}
    R = torch.einsum('iz,jz->ij', basex, q)   # (3,3)

    # 5. 计算平移向量 t，使得 R·bias2 + t = bias1  ⇒  t = bias1 - R·bias2
    t = bias1 - (R @ bias2)                  # (3,)

    # 6. 构造 rama 参数 (L2, 12)：前 9 维放 R.flatten，后 3 维放 t
    L2 = coor2.shape[0]
    rama = torch.empty((L2, 12), dtype=torch.float64, device=coor2.device)
    R_flat = R.reshape(1, 9).repeat(L2, 1)    # (L2,9)
    t_rep  = t.reshape(1, 3).repeat(L2, 1)    # (L2,3)
    rama[:, :9] = R_flat
    rama[:, 9:] = t_rep

    # 7. 调用 quat2b：对 out2 整条链做刚性变换
    #    输入 coord=(L2,3,3)， rama=(L2,12)，输出同 shape
    coor2_aligned = a2b.quat2b(coor2, rama)   # torch.Tensor (L2,3,3)

    # 8. 拼接（跳过 coor2_aligned[0]，因为它已与 coor1[-1] 重合）
    coor_cat = torch.cat([coor1, coor2_aligned[1:]], dim=0)  # (L1+L2-1,3,3)

    return coor_cat.cpu().numpy()

In [8]:
def Get_base(seq, basenpy_standard):
    n_atoms = basenpy_standard.shape[1]
    basenpy = np.zeros([len(seq), n_atoms, 3])
    seqnpy = np.array(list(seq))
    basenpy[seqnpy=='A'] = basenpy_standard[0]
    basenpy[seqnpy=='a'] = basenpy_standard[0]
    basenpy[seqnpy=='G'] = basenpy_standard[1]
    basenpy[seqnpy=='g'] = basenpy_standard[1]
    basenpy[seqnpy=='C'] = basenpy_standard[2]
    basenpy[seqnpy=='c'] = basenpy_standard[2]
    basenpy[seqnpy=='U'] = basenpy_standard[3]
    basenpy[seqnpy=='u'] = basenpy_standard[3]
    basenpy[seqnpy=='T'] = basenpy_standard[3]
    basenpy[seqnpy=='t'] = basenpy_standard[3]
    return basenpy

In [9]:
import numpy as np

def score_energy_one_simple(seq, target_id, out):
    """
    计算 RNA 结构的粗粒化自由能，只基于每个碱基的 P–C4'–N (N1 or N9) 坐标帧。

    参数：
      seq:      RNA 序列字符串，长度 L
      target_id:结构标识符（仅用于日志或输出，不影响计算）
      out:      模型输出字典，必须包含：
                out['coor']: numpy array, shape (L, 3, 3)
                             coor[i,0] = P 坐标
                             coor[i,1] = C4′ 坐标
                             coor[i,2] = N1/N9 坐标

    返回：
      total_energy: float，自由能估计值（越低表示越稳定）
    """

    coor = out['coor']  # (L, 3, 3)
    L = len(seq)

    # —— 一、键长能 —— #
    # 理想键长 (Å)：P–C4′ ~ 1.60, C4′–N ~ 1.47 (典型值，可根据经验或晶体数据调整)
    d0_P_C4  = 1.60
    d0_C4_N  = 1.47
    k_bond   = 100.0  # 键长力常数 (kcal/mol/Å²)

    energy_bond = 0.0
    for i in range(L):
        P  = coor[i,0]
        C4 = coor[i,1]
        N  = coor[i,2]
        d_PC4 = np.linalg.norm(P - C4)
        d_C4N = np.linalg.norm(C4 - N)
        energy_bond += k_bond * (d_PC4 - d0_P_C4)**2
        energy_bond += k_bond * (d_C4N - d0_C4_N)**2

    # —— 二、键角能 —— #
    #  ideal angle P–C4′–N ~ 109.5°（四面体角），用简谐势
    theta0   = np.deg2rad(109.5)
    k_angle  = 20.0   # 键角力常数 (kcal/mol/rad²)

    energy_angle = 0.0
    for i in range(L):
        P  = coor[i,0]
        C4 = coor[i,1]
        N  = coor[i,2]
        v1 = P  - C4
        v2 = N  - C4
        cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1)*np.linalg.norm(v2) + 1e-8)
        theta = np.arccos(np.clip(cos_theta, -1.0, 1.0))
        energy_angle += k_angle * (theta - theta0)**2

    # —— 三、堆积相互作用 —— #
    # 简单地对相邻碱基 C4′–C4′ 间距进行势能惩罚，鼓励相邻堆积：
    #   ideal 平面内距离 ~ 3.4 Å
    d0_stack = 3.4
    k_stack  = 5.0   # (kcal/mol/Å²)

    energy_stack = 0.0
    for i in range(L-1):
        C4_i   = coor[i  ,1]
        C4_ip1 = coor[i+1,1]
        d = np.linalg.norm(C4_i - C4_ip1)
        energy_stack += k_stack * (d - d0_stack)**2

    # —— 四、合计 —— #
    total_energy = energy_bond + energy_angle + energy_stack

    # （可选）打印调试
    # print(f"[{target_id}] bond={energy_bond:.2f}, angle={energy_angle:.2f}, stack={energy_stack:.2f} → total={total_energy:.2f}")

    return total_energy

In [10]:
import numpy as np

def score_energy_one_full(seq, target_id, out, paired=None,
                               # stacking parameters (optional if needed)
                               d0_stack=3.4, k_stack=5.0,
                               # dihedral parameters
                               phi0=np.deg2rad(180.0), k_dihedral=5.0,
                               # hydrogen-bond parameters
                               d0_hb=2.9, k_hb=10.0,
                               # Lennard-Jones parameters
                               sigma=4.0, epsilon=0.1,
                               # implicit solvation (ASA) coefficient
                               k_solv=0.05,
                               # Debye-Hückel electrostatics
                               q_P=-1.0, epsilon_r=80.0, kappa=10.0,
                               k_e=332.0637,
                               # statistical potential map: dict of (i,j)->energy
                               stat_potentials=None,
                               # numerical stability
                               r_min=1.0):
    """
    以 TM-score 最大化为目的的粗粒化能量函数，包含：
      - 主链二面角 (backbone torsion)
      - 氢键 (hydrogen bonding)
      - 范德华 + 隐式溶剂 (LJ + solvation)
      - 静电相互作用 (electrostatics)
      - 知识库统计势 (knowledge-based potentials)
    已移除键长和键角项。
    """
    coor = out['coor']
    L = len(seq)
    if paired is None:
        paired = out.get('paired', [])

    # —— 一、主链二面角能 —— #
    def torsion_angle(a, b, c, d):
        b1, b2, b3 = b - a, c - b, d - c
        n1 = np.cross(b1, b2); n2 = np.cross(b2, b3)
        n1 /= (np.linalg.norm(n1) + 1e-8); n2 /= (np.linalg.norm(n2) + 1e-8)
        cos_phi = np.dot(n1, n2)
        return np.arccos(np.clip(cos_phi, -1, 1))

    E_dihedral = 0.0
    for i in range(L - 1):
        a, b, c = coor[i,0], coor[i,1], coor[i,2]
        d = coor[i+1,0]
        phi = torsion_angle(a, b, c, d)
        E_dihedral += k_dihedral * (phi - phi0)**2

    # —— 二、氢键势 —— #
    E_hb = 0.0
    for i, j in paired:
        r = np.linalg.norm(coor[i,2] - coor[j,2])
        E_hb += k_hb * (r - d0_hb)**2

    # —— 三、范德华 + 隐式溶剂 —— #
    # E_LJ = 0.0
    # E_solv = 0.0
    # for i in range(L):
    #     for j in range(i + 2, L):
    #         r = np.linalg.norm(coor[i,1] - coor[j,1])
    #         r_clamped = max(r, r_min)
    #         sr6 = (sigma / r_clamped)**6
    #         sr12 = sr6 * sr6
            # E_LJ += 4 * epsilon * (sr12 - sr6)
            # 隐式溶剂: 与 ASA 成正比, 这里简化使用 r^-2
            # E_solv += k_solv / (r_clamped**2)

    # —— 四、静电（Debye-Hückel）—— #
    E_elec = 0.0
    prefac = k_e * q_P * q_P / epsilon_r
    for i in range(L):
        for j in range(i + 1, L):
            r = np.linalg.norm(coor[i,0] - coor[j,0])
            r_clamped = max(r, r_min)
            E_elec += prefac * np.exp(-r_clamped / kappa) / r_clamped

    # —— 五、知识库统计势 —— #
    E_stat = 0.0
    if stat_potentials:
        for (i, j), Eij in stat_potentials.items():
            E_stat += Eij

    # total_energy = E_dihedral + E_hb + E_LJ + E_solv + E_elec + E_stat
    total_energy = E_dihedral + E_hb + E_elec + E_stat

    # print(f"[{target_id}] torsion={E_dihedral:.2f}, hb={E_hb:.2f}, LJ={E_LJ:.2f}, "
    #       f"solv={E_solv:.2f}, elec={E_elec:.2f}, stat={E_stat:.2f} -> total={total_energy:.2f}")
    # print(f"[{target_id}] torsion={E_dihedral:.2f}, hb={E_hb:.2f}, "
    #       f"elec={E_elec:.2f}, stat={E_stat:.2f} -> total={total_energy:.2f}")
    
    return -total_energy

In [11]:
import pyrosetta
from pyrosetta import pose_from_pdb, create_score_function
from pyrosetta.rosetta import core

# 1. 初始化 PyRosetta（只需执行一次）
pyrosetta.init(options='-mute all')  # 静默模式，不输出多余日志

# 3. 创建并定制 ScoreFunction
op_score = create_score_function('ref2015')
# 3.1 约束权重
op_score.set_weight(core.scoring.atom_pair_constraint, 9.0)
op_score.set_weight(core.scoring.dihedral_constraint, 4.0)
op_score.set_weight(core.scoring.angle_constraint, 4.0)
# 3.2 物理/统计势能项权重
op_score.set_weight(core.scoring.fa_rep, 9.0)
op_score.set_weight(core.scoring.rna_sugar_close, 9.0)
op_score.set_weight(core.scoring.fa_intra_rep, 9.0)
op_score.set_weight(core.scoring.rna_base_pair, 9.0)
op_score.set_weight(core.scoring.rna_base_stack, 9.0)

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.Release.python310.ubuntu.cxx11thread.serialization 2025.13+release.80dd00bc09d2543ce4b9a50f13c49219ee5cdd3b 2025-03-27T08:32:50] retrieved from: http://www.pyrosetta.org


In [12]:
import os
import json
import pickle
from subprocess import PIPE, STDOUT, Popen
import numpy as np
import tempfile
from Bio.PDB import PDBParser
sys.path.append('/kaggle/input/drfold-model/DRfold2/PotentialFold')
from Optimization import Structure

def score_energy_one(seq, target_id, out):
    with tempfile.TemporaryDirectory() as tmpdirname:
        # 输出临时fasta
        fastafile = os.path.join(tmpdirname, 'tmp.fasta')
        with open(fastafile, 'w') as f:
            f.write(f'>{target_id}\n{seq}\n')
        # 输出临时ret
        retfile = os.path.join(tmpdirname, 'tmp.ret')
        with open(retfile, 'wb') as f:
            f.write(pickle.dumps(out))
        # foldconfig = '/kaggle/input/drfold-model/DRfold2/cfg_for_selection.json'
        # foldconfig = '/kaggle/input/drfold-model/DRfold2/cfg_for_folding.json'
        foldconfig = '/kaggle/input/drfold-model-bf16/cfg_for_folding.json'
        # foldconfig = 'cfg_for_folding.json'
        save_prefix = os.path.join(tmpdirname, 'tmp.json')
        stru=Structure(fastafile,[retfile],save_prefix,0,foldconfig)
        rama=stru.init_quat(0).data.numpy()
        energy=stru.obj_func_np(rama)
        return energy
    
def score_energy_one_op(seq, target_id, out):
    with tempfile.TemporaryDirectory() as tmpdirname:
        # 输出临时fasta
        fastafile = os.path.join(tmpdirname, 'tmp.fasta')
        with open(fastafile, 'w') as f:
            f.write(f'>{target_id}\n{seq}\n')
        # 输出临时ret
        retfile = os.path.join(tmpdirname, 'tmp.ret')
        with open(retfile, 'wb') as f:
            f.write(pickle.dumps(out))
        # foldconfig = '/kaggle/input/drfold-model/DRfold2/cfg_for_selection.json'
        # foldconfig = '/kaggle/input/drfold-model/DRfold2/cfg_for_folding.json'
        foldconfig = '/kaggle/input/drfold-model-bf16/cfg_for_folding.json'
        # foldconfig = 'cfg_for_folding.json'
        save_prefix = os.path.join(tmpdirname, 'tmp.json')
        stru=Structure(fastafile,[retfile],save_prefix,0,foldconfig)
        rama=stru.init_quat(0).data.numpy()
        
        cgpdb = os.path.join(tmpdirname, 'tmp.pdb')
        # print(rama)
        newrama=torch.DoubleTensor(rama) 
        stru.outpdb(newrama, cgpdb)
        
        savepdb = os.path.join(tmpdirname, 'tmp_full.pdb')
        
        exp_dir = os.getcwd()
        arena = os.path.join(exp_dir, 'Arena', 'Arena')
        cmd = f'{arena} {cgpdb} {savepdb} 7'
        p = Popen(cmd, shell=True, stdin=PIPE, stdout=PIPE, stderr=STDOUT)
        output, error = p.communicate()
        pose= pose_from_pdb(savepdb)
        
        score = op_score(pose)
        return score


def optimize_coor(seq, target_id, out):
    print('Optimizing structure for ', target_id)
    if len(out['coor'])>len(out['plddt']):
        # 补全plddt
        if len(out['plddt'])>0:
            mean_plddt = np.mean(out['plddt'])
            out['plddt'] = np.concatenate([out['plddt'], np.full((len(out['coor'])-len(out['plddt'])), mean_plddt)])
        else:
            out['plddt'] = np.full((len(out['coor'])), 0.0)
            
    if len(out['coor'])<len(out['plddt']):
        mean_plddt = np.mean(out['plddt'])
        out['plddt'] = np.full((len(out['coor'])), mean_plddt)
    
    with tempfile.TemporaryDirectory() as tmpdirname:
        # 1) 写入临时 fasta
        fastafile = os.path.join(tmpdirname, 'tmp.fasta')
        with open(fastafile, 'w') as f:
            f.write(f'>{target_id}\n{seq}\n')

        # 2) 写入临时 ret
        retfile = os.path.join(tmpdirname, 'tmp.ret')
        with open(retfile, 'wb') as f:
            pickle.dump(out, f)

        # 3) 调用 Structure 完成折叠优化
        foldconfig = '/kaggle/input/drfold-model/DRfold2/cfg_for_folding.json'
        # foldconfig = 'cfg_for_folding.json'
        save_prefix = os.path.join(tmpdirname, 'tmp')
        stru = Structure(fastafile, [retfile], save_prefix, 0, foldconfig)
        stru.foldning()

        # 4) 解析输出的 PDB
        pdb_file = save_prefix + '.pdb'
        parser = PDBParser(QUIET=True)
        structure = parser.get_structure(target_id, pdb_file)

        # 5) 收集所有标准残基（排除 HETATM）
        residues = [
            res for res in structure.get_residues()
            if res.id[0] == ' '
        ]
        # 检查残基数和序列长度一致
        L = len(seq)
        if len(residues) != L:
            raise ValueError(f"PDB 中残基数 ({len(residues)}) 与序列长度 ({L}) 不一致")

        # 6) 提取三种原子坐标
        atom_order = ['P', "C4'", 'N1/N9']
        coor = np.zeros((L, 3, 3), dtype=float)

        for i, res in enumerate(residues):
            coor[i, :, :] = np.nan  # 初始化为 NaN
            
            if 'P' in res:
                coord = res['P'].get_vector().get_array()
                coor[i, 0, :] = coord
            if "C4'" in res:
                # 缺失时填 NaN 方便排查
                coord = res["C4'"].get_vector().get_array()
                coor[i, 1, :] = coord
            if 'N1' in res:
            # 缺失时填 NaN 方便排查
                coord = res['N1'].get_vector().get_array()
                coor[i, 2, :] = coord
            elif 'N9' in res:
                # 缺失时填 NaN 方便排查
                coord = res['N9'].get_vector().get_array()
                coor[i, 2, :] = coord

        return coor

In [13]:
import sys
import os
import numpy as np
import sys
sys.path.append('/kaggle/input/drfold-model-bf16')
sys.path.append('/kaggle/input/drfold-model-bf16/PotentialFold')
# 添加自定义代码路径
sys.path.append(f'/kaggle/input/drfold-model-bf16/{CFG_DIR}')
sys.path.append(f'/kaggle/input/drfold-model-bf16/{CFG_DIR}/RNALM2')

BASE_COOR = np.load('/kaggle/input/drfold-model-bf16/PotentialFold/lib/base.npy')
OTHER_COOR = np.load('/kaggle/input/drfold-model-bf16/PotentialFold/lib/other2.npy')
SIDE_COOR = np.load('/kaggle/input/drfold-model-bf16/PotentialFold/lib/side.npy')


from EvoMSA2XYZ import MSA2XYZ
from RNALM2.Model import RNA2nd
from data import parse_seq



# data helper
def make_data(seq, device):
    aa_type = parse_seq(seq)
    base = Get_base(seq, BASE_COOR)
    seq_idx = np.arange(len(seq)) + 1

    msa = aa_type[None, :]
    msa = torch.from_numpy(msa)
    msa = torch.cat([msa, msa], 0)  # ???
    msa = F.one_hot(msa.long(), 6).float()

    base_x = torch.from_numpy(base).float()
    seq_idx = torch.from_numpy(seq_idx).long()

    msa, base_x, seq_idx = msa.to(device), base_x.to(device), seq_idx.to(device)
    return msa, base_x, seq_idx


def solution_to_submit_df(solution):
    submit_df = []
    for k,s in solution.items():
        df = coord_to_df(s.sequence, s.coord, s.target_id)
        submit_df.append(df)
    
    submit_df = pd.concat(submit_df)
    return submit_df
 

def coord_to_df(sequence, coord, target_id):
    L = len(sequence)
    df = pd.DataFrame()
    df['ID'] = [f'{target_id}_{i + 1}' for i in range(L)]
    df['resname'] = [s for s in sequence]
    df['resid'] = [i + 1 for i in range(L)]

    num_coord = len(coord)
    for j in range(num_coord):
        df[f'x_{j+1}'] = coord[j][:, 0]
        df[f'y_{j+1}'] = coord[j][:, 1]
        df[f'z_{j+1}'] = coord[j][:, 2]
    return df


out_dir = '/kaggle/working/model-output'
os.makedirs(out_dir, exist_ok=True)


/kaggle/input/drfold-model-bf16/cfg_97/EvoMSA2XYZ.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  RNAlm.load_state_dict(torch.load(saved_model,map_location=torch.device

In [14]:
import warnings
warnings.filterwarnings("ignore")

def run_submit(valid_df):
    
    #load model (these are moified versions, not the same from their github repo)
    rnalm = RNA2nd(dict(
        s_in_dim=5,
        z_in_dim=2,
        s_dim= 512,
        z_dim= 128,
        N_elayers=18,
    ))
    rnalm_file = '/kaggle/input/drfold-model-bf16/model_hub/RCLM/epoch_67000'
    print(rnalm_file)
    print(
        rnalm.load_state_dict(torch.load(rnalm_file, map_location='cpu', weights_only=True), strict=False)
        #Unexpected key(s) in state_dict: "ss_head.linear.weight", "ss_head.linear.bias".
    )
    rnalm = rnalm.to(DEVICE)
    if(PREC=='fp16'):
        rnalm=rnalm.half()
        
    if PREC=='bf16':
        rnalm = rnalm.bfloat16()
        
    rnalm = rnalm.eval()

    #---
    msa2xyz = MSA2XYZ(
        seq_dim=6,
        msa_dim=7,
        N_ensemble=1,
        N_cycle=8,  # 8
        m_dim=64,
        s_dim=64,
        z_dim=64,
    )
    msa2xyz_file = [f'/kaggle/input/drfold-model-bf16/model_hub/{CFG_DIR}/model_{i}' for i in range(20)]
    if CFG_MERGE:
        msa2xyz_file = [
            f'/kaggle/input/drfold-model-bf16/model_hub/cfg_97/model_{i}'
            for i in range(20)
        ] + [
            f'/kaggle/input/drfold-model-bf16/model_hub/cfg_95/model_{i}'
            for i in range(20)
        ] + [
            f'/kaggle/input/drfold-model-bf16/model_hub/cfg_96/model_{i}'
            for i in range(20)
        ] + [
            f'/kaggle/input/drfold-model-bf16/model_hub/cfg_99/model_{i}'
            for i in range(20)
        ]
    num_msa2xyz = len(msa2xyz_file) 
    msa2xyz_state_dict = []
    for c in range(num_msa2xyz):
        if c==0: print(msa2xyz_file[c])
        m = torch.load(msa2xyz_file[c], map_location='cpu', weights_only=True)
        msa2xyz_state_dict.append(m)
        
    #print(msa2xyz.load_state_dict(msa2xyz_state_dict[0], strict=True))
    print(msa2xyz.load_state_dict(msa2xyz_state_dict[0], strict=False))
    msa2xyz = msa2xyz.to(DEVICE)
    if(PREC=='fp16'):
        msa2xyz=msa2xyz.half()

    if PREC=='bf16':
        msa2xyz = msa2xyz.bfloat16()
    msa2xyz = msa2xyz.eval()
    
    msa2xyz.msaxyzone.premsa.rnalm = rnalm

    #---
    # start here !!!!!!!!!!!!!!!!!!!!!!
    #valid_df = valid_df.iloc[[0,1]].reset_index(drop=True)


    submit_df = [] 
    total_time_taken = 0
    max_gpu_mem_used = 0

    for i, row in valid_df.iterrows():
        start_timer = timer()
        target_id = row.target_id  # 'R1116' #casp15 R1116: len(157)
        sequence = row.sequence
        seq = row.sequence  
        L = len(seq)
        if L > MAX_CAT_LENGTH:
            seq = seq[:MAX_CAT_LENGTH]
        # else:
        #     continue
        print(i, target_id, L, len(seq), seq[:75] + '...')

        
        if len(seq)>480:
            model_to_try=[16, 9, 1, 2, 0]
        elif len(seq)>200:
            # model_to_try = [0,1,2,8,9]
            model_to_try = [13, 6, 14, 5, 3]
        elif  len(seq)>100:
            # model_to_try = [0,2,4,6,8,10,12,14,16,18]#list(range(min(num_msa2xyz,10)))
            model_to_try = [13, 6, 14, 12, 7, 2, 5, 19, 10, 9]
            if CFG_MERGE:
                model_to_try = [24, 34, 20, 13, 6, 37, 28, 25, 14, 39]
        else:
            # model_to_try = list(range(num_msa2xyz))
            
            # model_to_try = list(range(20))
            model_to_try = [1, 2, 0, 8, 7, 5, 6, 14, 10, 18, 4, 13, 3, 17, 19, 11, 12, 15, 16, 9]
            
            # if CFG_MERGE:
            #     model_to_try = list(range(20)) + list(range(40,60)) + list(range(60, 80))
            
        if NO_SORT:
            model_to_try=model_to_try[:5]


        # 分段预测
        def predict_segment(seq):
            msa, base_x, seq_idx = make_data(seq, DEVICE)
            with torch.no_grad():
                if PREC=='fp16':
                    msa, base_x = msa.half(), base_x.half()
                if PREC=='bf16':
                    msa, base_x = msa.bfloat16(), base_x.bfloat16()
                return msa2xyz.pred(msa, seq_idx, None, base_x, np.array(list(seq)))

                
        energy = []
        coordinate=[]
        outputs=[]
        for c in model_to_try:
            msa2xyz.load_state_dict(msa2xyz_state_dict[c], strict=False)

            if len(seq) <= MAX_LENGTH:
                outs = [ predict_segment(seq) ]
            else:
                step = MAX_LENGTH - 1
                outs = []
                for s in range(0, len(seq), step):
                    seg = seq[s : min(s+MAX_LENGTH, len(seq))]
                    outs.append(predict_segment(seg))
                    
            out_cat = outs[0]
            for out_seg in outs[1:]:
                out_cat = {'coor': concat_coor(out_cat, out_seg)}
                    
            if NO_SORT:
                e=0
            elif len(model_to_try)>5 and DR_SCORE:
                e = score_energy_one(seq, target_id, out_cat)
            elif len(model_to_try)>5 and OP_SCORE:
                e = score_energy_one_op(seq, target_id, out_cat)
            elif FULL_ENERGY:
                e = score_energy_one_full(seq, target_id, out_cat)
            else:
                e = score_energy_one_simple(seq, target_id, out_cat)
            energy.append(e) #tranucated sequence
            
            if L != len(seq):
                out_cat['coor'] = np.pad(out_cat['coor'], ((0, L - len(seq)), (0, 0), (0, 0)), 'constant', constant_values=0)
                
            outputs.append(out_cat)
            
            
            xyz = frame_coor_to_C1(out_cat['coor'], sequence, BASE_COOR, OTHER_COOR)
            
            coordinate.append(xyz)
            

            time_taken = timer() - start_timer
            total_time_taken += time_taken
            #print('time_taken:', time_to_str(time_taken, mode='sec'))

            gpu_mem_used = gpu_memory_use()
            max_gpu_mem_used = max(max_gpu_mem_used,gpu_mem_used)
            #print('gpu_mem_used:', gpu_mem_used, 'GB')

            print(f'{c:02d}   energy:{e:10.0f}   out_cat{str(out_cat["coor"].shape)}  time:{time_to_str(time_taken, mode="sec")}   gpu={gpu_mem_used} gb')

            
        #------- 
        torch.cuda.empty_cache()
        
        if GET_CENTER:
            energy = np.array(energy)
            energy_mean = np.mean(energy)
            energy= np.abs(energy - energy_mean)
        #select top5
        argsort = np.array(energy).argsort()
        argsort = argsort[:5]
        
        if L <= OPTIM_LENGTH:
            out_opt= outputs[argsort[0]]
            out_opt['coor'] = optimize_coor(seq, target_id, out_opt)
            coordinate[argsort[0]] = frame_coor_to_C1(out_opt['coor'], sequence, BASE_COOR, OTHER_COOR)
            torch.cuda.empty_cache()
            
        df = coord_to_df(row.sequence, [coordinate[k] for k in argsort], row.target_id)
        submit_df.append(df)
    
    print('----------------------------------------')
    print('MAX_LENGTH', MAX_LENGTH)
    print('### total_time_taken:', time_to_str(total_time_taken, mode='min'))
    print('### max_gpu_mem_used:', max_gpu_mem_used, 'GB')
    print('')

    submit_df = pd.concat(submit_df)
    submit_df.to_csv(f'submission.csv', index=False)
    print(submit_df)
    return submit_df

run_submit(valid_df)

print('SUBMIT OK!!!')

/kaggle/input/drfold-model-bf16/model_hub/RCLM/epoch_67000
_IncompatibleKeys(missing_keys=[], unexpected_keys=['ss_head.linear.weight', 'ss_head.linear.bias'])
/kaggle/input/drfold-model-bf16/model_hub/cfg_97/model_0
<All keys matched successfully>
0 R1107 69 69 GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUUGCACUCCGGCUGCGAAUUCUGCU...
fitting cubic distance
fitting cubic distance
fitting cubic distance
01   energy:     95925   out_cat(69, 3, 3)  time: 0 min 10 sec   gpu=1.76 gb
fitting cubic distance
fitting cubic distance
fitting cubic distance
02   energy:     80128   out_cat(69, 3, 3)  time: 0 min 18 sec   gpu=1.76 gb
fitting cubic distance
fitting cubic distance
fitting cubic distance
00   energy:     42217   out_cat(69, 3, 3)  time: 0 min 25 sec   gpu=1.76 gb
fitting cubic distance
fitting cubic distance
fitting cubic distance
08   energy:    137119   out_cat(69, 3, 3)  time: 0 min 33 sec   gpu=1.76 gb
fitting cubic distance
fitting cubic distance
fitting cubic distance
07   energy

In [15]:
display(pd.read_csv('submission.csv'))

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
0,R1107_1,G,1,4.845157,-0.634601,-0.185534,1.157847,-3.525067,1.703459,1.728291,10.014613,-1.497908,0.685246,-2.728395,1.066493,-0.361686,1.297108,-3.245297
1,R1107_2,G,2,1.545155,-3.028500,-2.619719,-1.318608,-5.860489,-2.291032,4.743384,12.862622,1.562650,-3.431532,-4.192187,-1.146264,2.287350,5.190075,-2.426551
2,R1107_3,G,3,-3.181917,-3.887024,-3.711864,-5.935173,-7.224083,-4.575875,7.513202,13.253139,5.966415,-8.252941,-3.372105,-1.718915,3.688397,8.967188,0.395733
3,R1107_4,G,4,-8.283877,-2.356641,-3.764495,-11.381451,-6.754535,-5.106342,9.752652,10.884346,9.971132,-12.539780,-0.600036,-1.314690,3.857205,11.599137,4.820839
4,R1107_5,G,5,-12.164174,1.260459,-3.313392,-16.282185,-4.301196,-4.330557,11.314575,7.220200,13.106038,-15.357626,3.658454,-0.459829,3.105890,12.367591,10.049135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,R1190_114,U,114,24.648096,-1.502128,-34.269722,15.021949,44.537148,-6.660552,18.873993,48.507214,3.232740,-20.412876,21.216380,-26.073614,9.991468,40.535250,-27.662706
2511,R1190_115,U,115,24.909388,1.643626,-38.439780,11.007890,43.266003,-9.711420,17.947954,51.130450,0.350007,-22.256880,21.077730,-31.199787,6.586556,42.664265,-29.540726
2512,R1190_116,U,116,24.601260,2.859881,-43.179480,8.258856,40.472637,-12.948432,17.418890,51.027134,-3.187981,-26.068375,20.962723,-34.666480,3.218835,43.535637,-31.652658
2513,R1190_117,U,117,22.733600,1.562139,-47.911900,6.968361,35.863216,-14.699681,14.460673,49.911533,-5.293249,-31.136356,22.043312,-35.687010,-0.163253,41.864704,-33.967354
